# Particle Set


Simulate trajectories of a particle cloud in a two-dimensional flow field.
A doubly-periodic domain and randomly-generated flow fields are initially used.
For additional documentation e.g. see :
[1](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/),
[2](https://JuliaClimate.github.io/MeshArrays.jl/dev/),
[3](https://docs.juliadiffeq.org/latest/solvers/ode_solve.html),
[4](https://en.wikipedia.org/wiki/Displacement_(vector))

Exercises:
- change the initial distribution of partices
- increase the duration of the trajectories simulation
- treat the non-periodic domain case by padding `u,v` with zeros
- replace `u,v` with your own two-dimensional flow fields

![particles in random flow](https://github.com/JuliaClimate/IndividualDisplacements.jl/raw/master/examples/figs/RandomFlow.gif)

## 1. Import Software

In [1]:
using IndividualDisplacements, DataFrames
p=dirname(pathof(IndividualDisplacements))
include(joinpath(p,"../examples/helper_functions.jl"));

## 2. Flow Fields

In [2]:
u,v,ϕ=setup_random_flow();

The above `u,v` arrays can be replaced with any other pair provided by the user.

A couple of important considerations, however:

- `u,v` are staggered on a C-grid; by `-0.5` grid point in direction `1` for `u` (`2` for `v`)
 from the grid cell center (0.5,0.5)
- `u,v` here derive from streamfunction `ϕ`, defined at the corner point, which ensures that
 the resulting `u,v` is non-divergent, purely rotational, over the C-grid domain.
In brief:

```
u=-(circshift(ϕ, (0,-1))-ϕ)
v=(circshift(ϕ, (-1,0))-ϕ)
```

If user were to start with collocated velocity (`uC,vC` at the grid cell center) then
one can easily obtain the staggered velocity (`u,v`) as follows. These may contain both
[rotational and divergent](https://en.wikipedia.org/wiki/Helmholtz_decomposition) components.

```
u=0.5*(circshift(uC, (0,1))+uC)
v=0.5*(circshift(vC, (1,0))+vC)
```

A convenient way to set up the flow fields using the MeshArrays.jl package (which
handles such staggered grids in general fashion) is to call `setup_F_MeshArray2D`

In [3]:
𝐹=setup_F_MeshArray2D(u,v);

## 3. Initialize Individuals

For example, we can initialize 100 particles within a central subdomain as follows.

In [4]:
np,nq=size(u)
x=np*(0. .+ 1.0*rand(1000))
y=nq*(0. .+ 1.0*rand(1000))
a=ones(size(x)); #subdomain array index (just 1 here)

The following constructor function wraps everything in the `Individuals` data structure.

In [5]:
𝐼=Individuals(𝐹,x,y,a)

  📌 details     = (1, 1000) Array{Float64,1}
  🔴 details     = (0, 5) ["ID", "x", "y", "fid", "t"]
  🆔 range       = (1, 1000)
  🚄 function    = dxy_dt!
  ∫  function    = solver_default
  🔧 function    = postprocess_MeshArray
  𝑃  details     = (:u0, :u1, :v0, :v1, :𝑇, :update_location!)


## 3. Compute Trajectories

The time period is `𝐼.𝑃.𝑇` by default, unless `∫!(𝐼,𝑇)` is called instead.

In [6]:
∫!(𝐼)

1×1000 Array{Array{Float64,1},2}:
 [2.96384, 7.5292, 1.0]  [7.38549, 5.94821, 1.0]  …  [11.013, 11.5051, 1.0]

## 4. Plot Results

For example, generate a simple animation:

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*